# Building ML Classifiers: Basic Random Forest & Gradient Boosting models

### Reading & cleaning text

In [27]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation]) # Counting puntuations
    return round(count/(len(text) - text.count(" ")), 3)*100  # Percentage of punctuations

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation]) # removing puntuations and creating list
    tokens = re.split('\W+', text) # removing multiple special characters --> '\W+' ('\W' -> single special character)
    #tokens = re.findall('\w+', text)  # alternative to above statement (Split -> '\W+'; findall -> '\w+')
    text = [ps.stem(word) for word in tokens if word not in stopwords]  # stemmig the words
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
print(X_features.head())

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)
print(X_count_feat.head())

   body_len  punct%    0    1    2    3    4    5    6    7  ...   8094  8095  \
0       128     4.7  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   0.0   
1        49     4.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   0.0   
2        62     3.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   0.0   
3        28     7.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   0.0   
4       135     4.4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0   0.0   

   8096  8097  8098  8099  8100  8101  8102  8103  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 rows x 8106 columns]
   body_len  punct%  0  1  2  3  4  5  6  7  ...   8094  8095  8096  8097  \
0       128     4.7  0  0  0  0  0  0  0  0  ...      0     0     0     0   
1        49     4.1  0

### Exploring RandomForestClassifier Attributes & Hyperparameters

In [2]:
from sklearn.ensemble import RandomForestClassifier

In [3]:
print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_estimator_type', '_get_param_names', '_make_estimator', '_set_oob_score', '_validate_X_predict', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=N

### Exploing RandomForestClassifier through Cross-Validation

In [4]:
from sklearn.model_selection import KFold, cross_val_score

In [5]:
rf_kfold = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)
cross_val_score(rf_kfold, X_features, data['label'], cv=k_fold, scoring='accuracy', n_jobs=-1)

array([0.97307002, 0.97127469, 0.96945193, 0.95956873, 0.96226415])

### Exploring RandomForestClassifier through holdout set

In [6]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [8]:
rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf.fit(X_train, y_train)

In [9]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[0:10]

[(0.05372391412142566, 'body_len'),
 (0.043465328102913106, 7350),
 (0.03525865295091706, 3134),
 (0.033836731937353204, 1803),
 (0.027930153898490895, 5724),
 (0.027704111988416488, 4796),
 (0.022141794137644277, 7027),
 (0.021302953942960688, 2031),
 (0.016993465753887563, 6746),
 (0.01693419866569699, 5078)]

In [10]:
y_pred = rf_model.predict(X_test)
precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')

In [11]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3),
                                                        round(recall, 3),
                                                        round((y_pred==y_test).sum() / len(y_pred),3)))

Precision: 1.0 / Recall: 0.592 / Accuracy: 0.936


### Building a Grid Search

In [16]:
def train_RF(n_est, depth):
    rf1 = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model1 = rf.fit(X_train, y_train)
    y_pred1 = rf_model.predict(X_test)
    precision1, recall1, fscore1, support1 = score(y_test, y_pred1, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        n_est, depth, round(precision1, 3), round(recall1, 3),
        round((y_pred1==y_test).sum() / len(y_pred1), 3)))

In [17]:
for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)

Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.58 / Accuracy: 0.934
Est: 10 / Depth: 20 ---- Precision: 1.0 / Recall: 0.557 / Accuracy: 0.931
Est: 10 / Depth: 30 ---- Precision: 1.0 / Recall: 0.557 / Accuracy: 0.931
Est: 10 / Depth: None ---- Precision: 1.0 / Recall: 0.557 / Accuracy: 0.931
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 0.534 / Accuracy: 0.927
Est: 50 / Depth: 20 ---- Precision: 1.0 / Recall: 0.517 / Accuracy: 0.925
Est: 50 / Depth: 30 ---- Precision: 1.0 / Recall: 0.54 / Accuracy: 0.928
Est: 50 / Depth: None ---- Precision: 1.0 / Recall: 0.563 / Accuracy: 0.932
Est: 100 / Depth: 10 ---- Precision: 1.0 / Recall: 0.58 / Accuracy: 0.934
Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 0.575 / Accuracy: 0.934
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 0.58 / Accuracy: 0.934
Est: 100 / Depth: None ---- Precision: 1.0 / Recall: 0.552 / Accuracy: 0.93


### Evaluating Random forest with GridSearchCV

In [18]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [19]:
from sklearn.model_selection import GridSearchCV

In [21]:
rf2 = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf2, param, cv=5, n_jobs=-1)

In [23]:
gs_fit1 = gs.fit(X_features, data['label'])
pd.DataFrame(gs_fit1.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

C:\Users\Abhi\Anaconda3\envs\py35\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Abhi\Anaconda3\envs\py35\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Abhi\Anaconda3\envs\py35\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Abhi\Anaconda3\envs\p

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_max_depth,param_n_estimators,params,rank_test_score,split0_test_score,split0_train_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
8,44.529136,1.509081,0.974493,0.999281,90,300,"{'max_depth': 90, 'n_estimators': 300}",1,0.978475,0.999326,...,0.975741,0.999102,0.966757,0.999775,0.973944,0.998877,4.767877,0.307954,0.004168,0.000298
11,43.938558,1.210974,0.974133,1.000000,None,300,"{'max_depth': None, 'n_estimators': 300}",2,0.977578,1.000000,...,0.976640,1.000000,0.969452,1.000000,0.972147,1.000000,10.328994,0.543204,0.002984,0.000000
7,23.955285,0.933581,0.973954,0.999012,90,150,"{'max_depth': 90, 'n_estimators': 150}",3,0.978475,0.999102,...,0.972147,0.998877,0.966757,0.999102,0.974843,0.999102,0.678141,0.143061,0.004223,0.000110
10,25.886885,0.977440,0.973415,1.000000,None,150,"{'max_depth': None, 'n_estimators': 150}",4,0.979372,1.000000,...,0.975741,1.000000,0.968553,1.000000,0.969452,1.000000,0.988231,0.157695,0.004014,0.000000
6,5.542940,0.471497,0.973235,0.998294,90,10,"{'max_depth': 90, 'n_estimators': 10}",5,0.975785,0.998428,...,0.979335,0.997755,0.971249,0.998204,0.965858,0.998428,0.460693,0.095420,0.004529,0.000304


In [28]:
gs_fit2 = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit2.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

C:\Users\Abhi\Anaconda3\envs\py35\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Abhi\Anaconda3\envs\py35\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Abhi\Anaconda3\envs\py35\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Abhi\Anaconda3\envs\p

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_max_depth,param_n_estimators,params,rank_test_score,split0_test_score,split0_train_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
7,28.621162,1.044288,0.973415,0.998698,90,150,"{'max_depth': 90, 'n_estimators': 150}",1,0.977578,0.998877,...,0.975741,0.998653,0.966757,0.999102,0.971249,0.998428,4.887849,0.319068,0.003929,0.000262
11,50.391550,1.337137,0.973415,1.000000,None,300,"{'max_depth': None, 'n_estimators': 300}",1,0.977578,1.000000,...,0.973944,1.000000,0.967655,1.000000,0.971249,1.000000,6.912391,0.514937,0.003630,0.000000
8,50.071497,1.361245,0.972157,0.998922,90,300,"{'max_depth': 90, 'n_estimators': 300}",3,0.976682,0.998652,...,0.973046,0.999102,0.966757,0.999102,0.971249,0.998653,7.316556,0.160921,0.003227,0.000220
10,27.836672,1.030481,0.971978,1.000000,None,150,"{'max_depth': None, 'n_estimators': 150}",4,0.977578,1.000000,...,0.973944,1.000000,0.969452,1.000000,0.969452,1.000000,0.792111,0.214806,0.003299,0.000000
4,23.637409,0.810223,0.971080,0.992770,60,150,"{'max_depth': 60, 'n_estimators': 150}",5,0.976682,0.992812,...,0.968553,0.993489,0.965858,0.993040,0.970350,0.992366,0.952002,0.191064,0.003842,0.000479


### Exploring GradientBoostingClassifier Attributes & Hyperparameters

In [29]:
from sklearn.ensemble import GradientBoostingClassifier

In [30]:
print(dir(GradientBoostingClassifier))
print(GradientBoostingClassifier())

['_SUPPORTED_LOSS', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_check_initialized', '_check_params', '_clear_state', '_decision_function', '_estimator_type', '_fit_stage', '_fit_stages', '_get_param_names', '_init_decision_function', '_init_state', '_is_initialized', '_make_estimator', '_resize_state', '_staged_decision_function', '_validate_estimator', '_validate_y', 'apply', 'decision_function', 'feature_importances_', 'fit', 'get_params', 'n_features', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params', 'staged_d

### Building a Grid Search for Gradient Boosting

In [31]:
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred2 = gb_model.predict(X_test)
    precision2, recall2, fscore2, train_support2 = score(y_test, y_pred2, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} / LR: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        est, max_depth, lr, round(precision2, 3), round(recall2, 3), 
        round((y_pred2==y_test).sum()/len(y_pred2), 3)))

In [32]:
for n_est in [50, 100, 150]:
    for max_depth in [3, 7, 11, 15]:
        for lr in [0.01, 0.1, 1]:
            train_GB(n_est, max_depth, lr)

C:\Users\Abhi\Anaconda3\envs\py35\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Est: 50 / Depth: 3 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.844
Est: 50 / Depth: 3 / LR: 0.1 ---- Precision: 0.947 / Recall: 0.713 / Accuracy: 0.949
Est: 50 / Depth: 3 / LR: 1 ---- Precision: 0.908 / Recall: 0.799 / Accuracy: 0.956
Est: 50 / Depth: 7 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.006 / Accuracy: 0.845
Est: 50 / Depth: 7 / LR: 0.1 ---- Precision: 0.929 / Recall: 0.833 / Accuracy: 0.964
Est: 50 / Depth: 7 / LR: 1 ---- Precision: 0.869 / Recall: 0.799 / Accuracy: 0.95
Est: 50 / Depth: 11 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.011 / Accuracy: 0.846
Est: 50 / Depth: 11 / LR: 0.1 ---- Precision: 0.929 / Recall: 0.833 / Accuracy: 0.964
Est: 50 / Depth: 11 / LR: 1 ---- Precision: 0.886 / Recall: 0.845 / Accuracy: 0.959
Est: 50 / Depth: 15 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.017 / Accuracy: 0.846
Est: 50 / Depth: 15 / LR: 0.1 ---- Precision: 0.925 / Recall: 0.845 / Accuracy: 0.965
Est: 50 / Depth: 15 / LR: 1 ---- Precision: 0.881 / Recall: 0.81 / Accura

KeyboardInterrupt: 

### Evaluating Gradient Boosting with GridSearchCV

In [ ]:
#from sklearn.model_selection import GridSearchCV

gb2 = GradientBoostingClassifier()
param = {
    'n_estimators': [100, 150], 
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb2, param, cv=5, n_jobs=-1)

In [ ]:
cv_fit = clf.fit(X_features, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

In [ ]:
cv_fit = clf.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]